In [1]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
import time
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.feature_selection import RFECV
from sklearn.model_selection import ShuffleSplit
import xgboost as xgb
from xgboost import plot_importance
from xgboost.sklearn import XGBRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression
import category_encoders as ce
import math
import datetime
import random
import tensorflow as tf

from numpy.random import seed




In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf


from numpy.random import seed

seed_value= 0
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)
seed(seed_value)
np.random.seed(seed_value)


In [2]:
def read_data(path):
    '''读取数据'''
    try:
        data = pd.read_csv(path, engine='python', encoding='utf8')
        return data
    except FileNotFoundError:
        print('File not found. Please check the path and filename.')
        return None
    except Exception as e:
        print('An error occurred:', e)
        return None

def assess_metrics(truth,pred):
    '''MAE、MSE、RMSE、决定系数'''
    print('----')
    print('MAE: %.4f'%mean_absolute_error(truth,pred))
    print('MSE: %.4f'%mean_squared_error(truth,pred))
    print('RMSE: %.4f'%mean_squared_error(truth,pred,squared=False))
    print('R2: %.4f'%r2_score(truth,pred))
    return mean_squared_error(truth,pred,squared=False),r2_score(truth,pred)

def model_result(est,X_train_val, X_test, y_train_val, y_test):
    '''Train and test a model, and print the results'''
    # training
    y_train_pred = est.predict(X_train_val)
    print('train metrics')
    assess_metrics(y_train_val, y_train_pred)
    # test
    print('test metrics')
    y_test_pred = est.predict(X_test)
    RMSE, R2 = assess_metrics(y_test, y_test_pred)
    return RMSE, R2
def model_lineresult(est,model,year,X_train_val, X_test, y_train_val, y_test):
    '''Train and test a linear regression model, and print the results in a line'''
    date = datetime.datetime.now().strftime('%Y-%m-%d')
    # train and test
    y_train_pred = est.predict(X_train_val)
    y_test_pred = est.predict(X_test)
    print('{} {} {} 【Train_RMSE: {:.2f}, Train_R2: {:.2f}, Test_RMSE: {:.4f}, Test_R2: {:.2f}】'
          .format(date,year,model,mean_squared_error(y_train_val, y_train_pred,squared=False),r2_score(y_train_val, y_train_pred),
          mean_squared_error(y_test, y_test_pred,squared=False),r2_score(y_test, y_test_pred)))
    return mean_squared_error(y_test, y_test_pred,squared=False),r2_score(y_test, y_test_pred)

### 大豆预测

In [3]:
# 大豆预测
# read data
# 2014-2021，每10年预测一年
data = read_data('D:/毕业大论文/数据/input_soybean_rename.csv')

# 删掉空缺值
data = data.dropna()

# 打印数据维度
print(data.shape)

# 打印特征列表区分时间特征、物候特征、静态特征
print(data.columns.tolist())

(12119, 1240)
['Year', 'State', 'Value', 'sta_con', 'yield(t/ha)', 'NDVI_Mid Mar.', 'EVI_Mid Mar.', 'LSWI_Mid Mar.', 'GCVI_Mid Mar.', 'RVI_Mid Mar.', 'SAVI_Mid Mar.', 'WDRVI_Mid Mar.', 'Fpar_Mid Mar.', 'LAI_Mid Mar.', 'ET_Mid Mar.', 'LE_Mid Mar.', 'LST_Day_Mid Mar.', 'LST_Night_Mid Mar.', 'SPI14d_Mid Mar.', 'SPI30d_Mid Mar.', 'SPI90d_Mid Mar.', 'EDDI14d_Mid Mar.', 'EDDI30d_Mid Mar.', 'EDDI90d_Mid Mar.', 'SPEI14d_Mid Mar.', 'SPEI30d_Mid Mar.', 'SPEI90d_Mid Mar.', 'PDSI_Mid Mar.', 'Palmer Z_Mid Mar.', 'Red_Mid Mar.', 'Nir_Mid Mar.', 'Blue_Mid Mar.', 'Green_Mid Mar.', 'Nir1_Mid Mar.', 'Swir1_Mid Mar.', 'Swir2_Mid Mar.', 'Precipitation_Mid Mar.', 'Temp_Mid Mar.', 'Humidity_Mid Mar.', 'Pressure_Mid Mar.', 'Shortwave_Mid Mar.', 'Longwave_Mid Mar.', 'NDVI_Late Mar.', 'EVI_Late Mar.', 'LSWI_Late Mar.', 'GCVI_Late Mar.', 'RVI_Late Mar.', 'SAVI_Late Mar.', 'WDRVI_Late Mar.', 'Fpar_Late Mar.', 'LAI_Late Mar.', 'ET_Late Mar.', 'LE_Late Mar.', 'LST_Day_Late Mar.', 'LST_Night_Late Mar.', 'SPI14d_Lat

In [ ]:
print('时间特征维度：',)
print('物候特征维度：')
print('静态特征维度：',len(['AREA_Irrigated', 'AREA', 'PIC', 
                     'CaCO3', 'CEC', 'Drainage', 'EC', 'I_class', 'N_class', 'Max_OM', 'PAWS', 'pH', 'SAR', 'Texture', 'Sand', 'Silt', 'Clay', 
                     'Longitude', 'Latitude', 
                     'IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']))
# 多余的应该删去的 'sta_con','State','yield(t/ha)','Year','Value','GDT_trend_yield'

In [9]:
import pandas as pd

# 构造一个示例DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': ['a', 'b', 'c'],
    'C': [True, False, True],
})

# 打印原始DataFrame
print('原始DataFrame：')
print(df)

# 定义新的列顺序
new_columns = ['C', 'A', 'B']

# 重新排列列顺序
df = df.reindex(columns=new_columns)

# 打印变换后的DataFrame
print('变换后的DataFrame：')
print(df)


原始DataFrame：
   A  B      C
0  1  a   True
1  2  b  False
2  3  c   True
变换后的DataFrame：
       C  A  B
0   True  1  a
1  False  2  b
2   True  3  c


In [10]:
str_list = data.columns.tolist()
sub_str_list = ['_Planting', '_Emerging', '_Blooming', '_Podding', '_Dropping leaves', '_Harvest']

result_list = [s for s in str_list if any(sub in s for sub in sub_str_list)]
# 按照顺序排序
order = ['_Planting', '_Emerging', '_Blooming', '_Podding', '_Dropping leaves', '_Harvest']

new_data = []
for o in order:
    for d in result_list:
        if o in d:
            new_data.append(d)

print(new_data)

['NDVI_Planting', 'EVI_Planting', 'LSWI_Planting', 'GCVI_Planting', 'RVI_Planting', 'SAVI_Planting', 'WDRVI_Planting', 'Fpar_Planting', 'LAI_Planting', 'LE_Planting', 'LST_Day_Planting', 'LST_Night_Planting', 'SPI14d_Planting', 'SPI30d_Planting', 'SPI90d_Planting', 'EDDI14d_Planting', 'EDDI30d_Planting', 'EDDI90d_Planting', 'SPEI14d_Planting', 'SPEI30d_Planting', 'SPEI90d_Planting', 'PDSI_Planting', 'Palmer Z_Planting', 'Red_Planting', 'Nir_Planting', 'Blue_Planting', 'Green_Planting', 'Nir1_Planting', 'Swir1_Planting', 'Swir2_Planting', 'Temp_Planting', 'Shortwave_Planting', 'Longwave_Planting', 'ET_Planting', 'NDVI_Emerging', 'EVI_Emerging', 'LSWI_Emerging', 'GCVI_Emerging', 'RVI_Emerging', 'SAVI_Emerging', 'WDRVI_Emerging', 'Fpar_Emerging', 'LAI_Emerging', 'LE_Emerging', 'LST_Day_Emerging', 'LST_Night_Emerging', 'SPI14d_Emerging', 'SPI30d_Emerging', 'SPI90d_Emerging', 'EDDI14d_Emerging', 'EDDI30d_Emerging', 'EDDI90d_Emerging', 'SPEI14d_Emerging', 'SPEI30d_Emerging', 'SPEI90d_Emergin

In [7]:
class LSTMModel(tf.keras.Model):
    def __init__(self, dynamic_shape, static_shape, lstm_units=16, dense_units=128, learning_rate=0.005):
        super(LSTMModel, self).__init__()

        self.lstm_units = lstm_units
        self.dense_units = dense_units

        self.dynamic_layer = tf.keras.layers.LSTM(units=lstm_units, return_sequences=False, input_shape=dynamic_shape)
        self.static_layer = tf.keras.layers.InputLayer(input_shape=static_shape)
        self.concat_layer = tf.keras.layers.Concatenate(axis=-1)
        self.dense_layer = tf.keras.layers.Dense(units=dense_units, activation='relu')
        self.output_layer = tf.keras.layers.Dense(units=1, activation='linear')

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.loss = tf.keras.losses.MeanSquaredError()

    def call(self, inputs):
        dynamic_inputs, static_inputs = inputs
        x = self.dynamic_layer(dynamic_inputs)
        x = self.concat_layer([x, static_inputs])
        x = self.dense_layer(x)
        output = self.output_layer(x)
        return output

# Reshape the data
X_train_dynamic = np.array(X_train_dynamic).reshape(10424,24,38)
X_train_static = np.array(X_train_static).reshape(10424,19)
y_train = np.array(y_train).reshape(10424,1)

X_val_dynamic = np.array(X_val_dynamic).reshape(1840,24,38)
X_val_static = np.array(X_val_static).reshape(1840,19)
y_val = np.array(y_val).reshape(1840,1)

X_test_dynamic = np.array(X_test_dynamic).reshape(775,24,38)
X_test_static = np.array(X_test_static).reshape(775,19)
y_test = np.array(y_test).reshape(775,1)

# Create the model
dynamic_shape = X_train_dynamic.shape[1:]
static_shape = X_train_static.shape[1:]
model = LSTMModel(dynamic_shape, static_shape)

# Compile the model
model.compile(optimizer=model.optimizer, loss=model.loss)

# Train the model
model.fit([X_train_dynamic, X_train_static], y_train, validation_data=([X_val_dynamic, X_val_static], y_val), epochs=50)

# Evaluate the model on the test set
model.evaluate([X_test_dynamic, X_test_static], y_test)


AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
def yield_predictor(data, test_year=[2020], N=2, experiment = 'GDT', croptype = 'soybean', model = ['XGBoost'], verbose = 0,
                    tune_model = False, model_parameter=None, result = False, result_dir = '', feature_importance = False,
                    fea_eng = False,seed = 99,save_model=False, save_model_dir='', trend_n = 30,
                    importance_dir = '', feature_list = None, n_features = 200, metric_list = []):
    '''Yield predictor
    Parameters:
    
    '''
    # split train-validation and test data
    # testset
    data_test = data[data['Year'].isin(test_year)]
    # tra-val set
    data_train_val = data[data['Year'].isin([i for i in range(test_year[0]-N,test_year[0])])]
    # split input and output
    y_train_val = data_train_val['yield(t/ha)']
    y_test = data_test['yield(t/ha)']
    
    if experiment == 'GDT': # GDT
        # de-trend the yield using linear model
        yield_mean = data_train_val['yield(t/ha)'].mean()
        linear_m = linear_model.LinearRegression()
        
        data_train_val1 = data[data['Year'].isin([i for i in range(test_year[0]-trend_n,test_year[0])])]
        X99 = np.array(data_train_val1['Year']).reshape(-1,1)
        Y99 = np.array(data_train_val1['yield(t/ha)']).reshape(-1,1)

        linear_m.fit(X99, Y99)
        if linear_m.coef_[0,0]<0:
            yield_mean = data_train_val['yield(t/ha)'].max()
        print('The foluma of de-trend the yield: yield_new = yield - {:.3f}×year + {:.3f} - ({:.3f})'
              .format(linear_m.coef_[0,0],yield_mean,linear_m.intercept_[0]))

        # de-trend data_train_val yield 
        data1 = data_train_val.copy()
        data1.loc[:,'yield(t/ha)_fix'] = data_train_val['yield(t/ha)'] - linear_m.coef_[0] * data_train_val['Year'] + yield_mean - linear_m.intercept_
        data1 = data1.drop(['yield(t/ha)'], axis = 1).rename({'yield(t/ha)_fix':'yield(t/ha)'}, axis=1)
        data_train_val = data1
        # de-trend data_test yield 
        data1 = data_test.copy()
        data1.loc[:,'yield(t/ha)_fix'] = data_test['yield(t/ha)'] - linear_m.coef_[0] * data_test['Year'] + yield_mean - linear_m.intercept_

        data1 = data1.drop(['yield(t/ha)'], axis = 1).rename({'yield(t/ha)_fix':'yield(t/ha)'}, axis=1)
        data_test = data1
        y_train_val = data_train_val['yield(t/ha)']
        y_test = data_test['yield(t/ha)']
        
        X_train_val = data_train_val.drop(['sta_con','State','yield(t/ha)','Year','Value','GDT_trend_yield'], axis=1)# 
        X_test = data_test.drop(['sta_con','State','yield(t/ha)','Year','Value','GDT_trend_yield'], axis=1) # 'Value'
    else: None
 
    # Scale numeric features
    columns_to_scale = X_train_val.columns.tolist()
    std_scaler = preprocessing.StandardScaler().fit(X_train_val[columns_to_scale])
    X_train_val.loc[:,columns_to_scale] = std_scaler.transform(X_train_val[columns_to_scale])
    X_test.loc[:,columns_to_scale] = std_scaler.transform(X_test[columns_to_scale])
    
    # 区分时间、物候、静态特征
    feature_time = 
    feature_phology = 
    feature_static = 
    

In [43]:
def yield_predictor(data, data1980, test_year = [2020], feature_list = [], detrend = True, usehistoy = True, model = ['XGBoost'], verbose = 0, shape=False,valsize=0.15,
                   tune_model = False, n_features = 500, model_parameter=None, learn_curve = False, result = False, feature_importance = False,
                   corr = False, rfecv = False,n_curve = False,Phenology = False,time = False, fea_eng = False):
    # Encoding categry feature
    # count encoding
    count_encoder = ce.count.CountEncoder(cols = ['_majority'],normalize = True).fit(data)
    data = count_encoder.transform(data)
#     # one-hot encoding
#     onehot_cols = ['state_name','Planted_Drought','Emergrd_Drought','Blooming_Drought','Setting Pods_Drought','Dropping Leaves_Drought','Harvested_Drought']
#     new_onehot_cols = pd.get_dummies(data[onehot_cols]).columns.tolist()
#     data.loc[:,new_onehot_cols] = pd.get_dummies(data[onehot_cols])
#     data = data.drop(onehot_cols,axis=1)
    # label encoding
    le = preprocessing.LabelEncoder()
    data.loc[:,'irrigation'] = le.fit_transform(data['irrigation'])
    data.loc[:,'state_name'] = le.fit_transform(data['state_name'])

    # split train-validation and test data
    data_test = data[data['year'].isin(test_year)]
    data_train_val = data[np.logical_not(data['year'].isin(test_year))]
    data1980 = data1980[np.logical_not(data1980['year'].isin(test_year))]

    if detrend:
        # de-trend the yield using linear model
        yield_mean = data_train_val['yield'].mean()
        linear_m = linear_model.LinearRegression()
        if usehistoy:
            X99 = np.array(data1980['year']).reshape(-1,1)
            Y99 = np.array(data1980['yield']).reshape(-1,1)
        else:
            X99 = np.array(data_train_val['year']).reshape(-1,1)
            Y99 = np.array(data_train_val['yield']).reshape(-1,1)
        linear_m.fit(X99, Y99)
    #         print('The foluma of de-trend the yield: yield_new = yield - {:.3f}×year + {:.3f} - ({:.3f})'
    #               .format(linear_m.coef_[0,0],yield_mean,linear_m.intercept_[0]))
        # de-trend data_train_val yield 
        data1 = data_train_val.copy()
        data1.loc[:,'yield_fix'] = data_train_val['yield'] - linear_m.coef_[0] * data_train_val['year'] + yield_mean - linear_m.intercept_
        data1 = data1.drop(['yield'], axis = 1).rename({'yield_fix':'yield'}, axis=1)
        data_train_val = data1
        # de-trend data_test yield 
        data1 = data_test.copy()
        data1.loc[:,'yield_fix'] = data_test['yield'] - linear_m.coef_[0] * data_test['year'] + yield_mean - linear_m.intercept_
        data1 = data1.drop(['yield'], axis = 1).rename({'yield_fix':'yield'}, axis=1)
        data_test = data1
    else: None

    # split input and output
    y_train_val = data_train_val['yield']
    y_test = data_test['yield']
    # print(data_train_val.head(5),y_train_val.head(5),data_test.head(5),y_test.head(5))
    X_train_val = data_train_val.drop(['Value', 'county_name', 'year', 'yield', 'sta_con','Planted_Drought', 'Emergrd_Drought', 'Blooming_Drought', 'Setting Pods_Drought', 'Dropping Leaves_Drought', 'Harvested_Drought'],axis=1)
    X_test = data_test.drop(['Value', 'county_name', 'year', 'yield', 'sta_con','Planted_Drought', 'Emergrd_Drought', 'Blooming_Drought', 'Setting Pods_Drought', 'Dropping Leaves_Drought', 'Harvested_Drought'],axis=1)
    # print(X_train_val.columns.tolist())  
    # Scale numeric features
    # not-numeric features list
    # print(data.columns.tolist())
#     not_numeric_list = data.columns.tolist()[-63:]
#     not_numeric_list.append('_majority')
#     not_numeric_list = [ele for ele in not_numeric_list if ele not in ['caco3_kg_sq_m','cec_025','ec_025','max_om','paws_025','ph_025','sar',
#                                                                        'sand_025','silt_025','clay_025']]
    columns_to_scale = X_train_val.columns.tolist()#[item for item in X_train_val.columns.tolist() if item not in not_numeric_list]
    std_scaler = preprocessing.MinMaxScaler().fit(X_train_val[columns_to_scale])# StandardScaler()
    X_train_val.loc[:,columns_to_scale] = std_scaler.transform(X_train_val[columns_to_scale])
    X_test.loc[:,columns_to_scale] = std_scaler.transform(X_test[columns_to_scale])
    # print(len(X_train_val.columns.tolist()))  
    #print(X_train_val.columns.tolist())
    # dynamic data
    all_features = X_train_val.columns.tolist()
    dynamic_features = []
    for i in range(9,33):
        for index in ['NDVI','EVI','LSWI','GCVI','RVI','SAVI','WDRVI','Fpar','Lai','ET','LE','LST_Day_1km','LST_Night_1km',
                      'spi14d','spi30d','spi90d','eddi14d','eddi30d','eddi90d','spei14d','spei30d','spei90d','pdsi','z',
                      'sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07',
                      'total_precipitation','temperature','specific_humidity','pressure','shortwave_radiation','longwave_radiation','temp_differ']:#
            dynamic_features.append(str(i)+'_'+index)
    X_train_val_dynamic = X_train_val[dynamic_features]
    X_test_dynamic = X_test[dynamic_features]
    # print(len(X_test_dynamic.columns.tolist()))
    # print(X_train_val.head(5))
    # print(X_train_val_dynamic.head(5))
    # static data
    X_train_val_static = X_train_val[['PIC', '_majority', 'x', 'y','irrigation', 'caco3_kg_sq_m', 'cec_025', 'drainage_class_int', 'ec_025', 'i_class',
                                     'n_class', 'max_om', 'paws_025', 'ph_025', 'sar', 'texture_025', 'sand_025', 'silt_025', 'clay_025']]
    X_test_static = X_test[['PIC', '_majority', 'x', 'y','irrigation', 'caco3_kg_sq_m', 'cec_025', 'drainage_class_int', 'ec_025', 'i_class',
                                     'n_class', 'max_om', 'paws_025', 'ph_025', 'sar', 'texture_025', 'sand_025', 'silt_025', 'clay_025']]
    # print(len(X_test_static.columns.tolist()))
    # print(X_train_val_static.head(5))
    #print(X_train_val_dynamic.shape,X_train_val_static.shape,X_test_dynamic.shape,X_test_static.shape,y_train_val.shape, y_test.shape)
    return X_train_val_dynamic,X_train_val_static,X_test_dynamic,X_test_static,y_train_val, y_test, yield_mean, linear_m.intercept_,linear_m.coef_[0]

In [7]:
X_train_val_dynamic,X_train_val_static,X_test_dynamic,X_test_static,y_train_val, y_test, yield_mean, slope = yield_predictor(data, data1980, test_year = [2020],  
                                                                                detrend=True, usehistoy=True, model=['XGBoost'],verbose=0,shape=False,)

In [8]:
X_train_val_dynamic

,9_NDVI,9_EVI,9_LSWI,9_GCVI,9_RVI,9_SAVI,9_WDRVI,9_Fpar,9_Lai,9_ET,...,32_sur_refl_b05,32_sur_refl_b06,32_sur_refl_b07,32_total_precipitation,32_temperature,32_specific_humidity,32_pressure,32_shortwave_radiation,32_longwave_radiation,32_temp_differ
0,0.389177,0.563946,0.145230,0.294910,0.135913,0.357349,0.194341,0.194636,0.091850,0.251403,...,0.511814,0.772097,0.936275,0.021905,0.683026,0.441730,0.789796,0.462955,0.560545,0.603385
1,0.411093,0.566342,0.240310,0.312766,0.148033,0.363829,0.210057,0.243893,0.107350,0.287634,...,0.389038,0.530964,0.576126,0.098688,0.633356,0.385406,0.781978,0.391996,0.521319,0.605832
2,0.389002,0.566642,0.159589,0.290140,0.136104,0.360389,0.194498,0.203026,0.096911,0.244545,...,0.507156,0.763803,0.903471,0.032405,0.671446,0.415459,0.796052,0.470395,0.543832,0.605457
3,0.372884,0.580892,0.295023,0.270268,0.127754,0.382120,0.183498,0.200115,0.089118,0.231119,...,0.447717,0.658913,0.749776,0.226818,0.592486,0.357373,0.726248,0.330696,0.474618,0.600239
4,0.398608,0.555181,0.117805,0.301546,0.140503,0.343695,0.200482,0.162029,0.070559,0.302861,...,0.412129,0.662385,0.831894,0.027437,0.667326,0.427147,0.766433,0.421325,0.539829,0.625811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13769,0.421940,0.577851,0.240349,0.316163,0.152742,0.375160,0.216572,0.277221,0.119788,0.413628,...,0.419818,0.500186,0.454273,0.106923,0.620622,0.449667,0.787565,0.313826,0.548886,0.549373
13770,0.453818,0.594876,0.294579,0.334383,0.170511,0.400367,0.239573,0.329443,0.152558,0.416827,...,0.449014,0.515810,0.476249,0.118764,0.620711,0.447980,0.791241,0.325473,0.552001,0.587888
13771,0.409460,0.580115,0.274076,0.316685,0.145916,0.378057,0.207684,0.257410,0.114728,0.422029,...,0.530324,0.604586,0.597367,0.146142,0.586232,0.415485,0.740834,0.261313,0.519156,0.603902
13772,0.483582,0.627266,0.327189,0.381001,0.187948,0.454391,0.261895,0.375952,0.181427,0.453863,...,0.479881,0.336928,0.295148,0.219759,0.576586,0.404974,0.731591,0.273936,0.518663,0.423990


In [9]:
y_test

13774    59.79863
13776    57.29863
13777    54.39863
13778    59.79863
13779    54.49863
           ...   
14634    45.69863
14635    48.59863
14636    49.79863
14637    48.49863
14638    42.59863
Name: yield, Length: 775, dtype: float64

In [10]:
X_train_dynamic, X_val_dynamic, y_train, y_val = train_test_split(X_train_val_dynamic,y_train_val, test_size=0.15, random_state= 99)
X_train_static, X_val_static, y_train, y_val = train_test_split(X_train_val_static,y_train_val, test_size=0.15, random_state= 99)
print(X_train_dynamic.shape,X_train_static.shape,y_train.shape,X_val_dynamic.shape,X_val_static.shape, y_val.shape,X_test_dynamic.shape,X_test_static.shape,y_test.shape)

(10424, 912) (10424, 19) (10424,) (1840, 912) (1840, 19) (1840,) (775, 912) (775, 19) (775,)


In [11]:
est = linear_model.LinearRegression()
est.fit(np.concatenate((X_train_dynamic,X_train_static),axis = -1), y_train)
print(test_metrics(y_val, est.predict(np.concatenate((X_val_dynamic,X_val_static),axis = -1))))
print(test_metrics(y_test, est.predict(np.concatenate((X_test_dynamic,X_test_static),axis = -1))))

Test_MAE: 2.8577
Test_MSE: 14.0479
Test_RMSE: 3.7480
Test_R2: 0.8496
----
None
Test_MAE: 6.4138
Test_MSE: 61.5298
Test_RMSE: 7.8441
Test_R2: 0.2766
----
None


In [584]:
# lstm
training_x_dynamic = np.array(X_train_dynamic).reshape(10424,24,38)
training_x_static = np.array(X_train_static).reshape(10424,19)
training_y = np.array(y_train).reshape(10424,1)

val_x_dynamic = np.array(X_val_dynamic).reshape(1840,24,38)
val_x_static = np.array(X_val_static).reshape(1840,19)
val_y = np.array(y_val).reshape(1840,1)

test_x_dynamic = np.array(X_test_dynamic).reshape(775,24,38)
test_x_static = np.array(X_test_static).reshape(775,19)
test_y = np.array(y_test).reshape(775,1)


unit = 16
inp1 = tf.keras.layers.Input(shape=(24,38))
inp2 = tf.keras.layers.Input(shape=(19,))
LSTM1 = tf.keras.layers.LSTM(units=unit, return_sequences=False)(inp1)
# LSTM2 = tf.keras.layers.LSTM(units=unit, return_sequences=True)(LSTM1)
# LSTM3 = tf.keras.layers.LSTM(units=unit, return_sequences=True)(LSTM2)
# LSTM4 = tf.keras.layers.LSTM(units=32, return_sequences=False)(LSTM1)
comb = tf.keras.layers.Concatenate(axis=-1)([LSTM1, inp2])
DENSE_1 = tf.keras.layers.Dense(units=128, activation='relu')(comb)
outp = tf.keras.layers.Dense(units=1, activation='linear')(DENSE_1)
model = tf.keras.Model([inp1, inp2], outp)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.005), loss=tf.keras.losses.MeanSquaredError())# 'mean_absolute_error'

In [581]:
# dnn
# (10424, 912) (10424, 19) (1840, 912) (1840, 19) (12264,) (1840,)
#(9415, 912) (9415, 19) (9415,) (1662, 912) (1662, 19) (1662,) (1962, 912) (1962, 19) (1962,)
training_x_dynamic = np.array(X_train_dynamic).reshape(X_train_dynamic.shape[0],912)#10424
training_x_static = np.array(X_train_static).reshape(X_train_dynamic.shape[0],19)
training_y = np.array(y_train).reshape(X_train_dynamic.shape[0],1)

val_x_dynamic = np.array(X_val_dynamic).reshape(X_val_dynamic.shape[0],912)#1840
val_x_static = np.array(X_val_static).reshape(X_val_dynamic.shape[0],19)
val_y = np.array(y_val).reshape(X_val_dynamic.shape[0],1)

test_x_dynamic = np.array(X_test_dynamic).reshape(X_test_dynamic.shape[0],912)# 
test_x_static = np.array(X_test_static).reshape(X_test_dynamic.shape[0],19)
test_y = np.array(y_test).reshape(X_test_dynamic.shape[0],1)

func_act = 'relu'
inp1 = tf.keras.layers.Input(shape=(912))
inp2 = tf.keras.layers.Input(shape=(19))
inp = tf.keras.layers.Concatenate(axis=-1)([inp1, inp2])
DENSE_0 = tf.keras.layers.Dense(units=512, activation=func_act)(inp)# ,kernel_regularizer=tf.keras.regularizers.l2(0.001)
# DENSE_01 = tf.keras.layers.Dropout(0.2)(DENSE_0)
DENSE_1 = tf.keras.layers.Dense(units=256, activation=func_act)(DENSE_0)#,kernel_regularizer=tf.keras.regularizers.l2(0.001)
DENSE_2 = tf.keras.layers.Dense(units=128, activation=func_act)(DENSE_1)
DENSE_3 = tf.keras.layers.Dense(units=64, activation=func_act)(DENSE_2)
DENSE_4 = tf.keras.layers.Dense(units=32, activation=func_act)(DENSE_3)
DENSE_5 = tf.keras.layers.Dense(units=16, activation=func_act)(DENSE_4)
DENSE_6 = tf.keras.layers.Dense(units=8, activation=func_act)(DENSE_5)
outp = tf.keras.layers.Dense(units=1, activation='linear')(DENSE_6)
model = tf.keras.Model([inp1, inp2], outp)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.MeanSquaredError())# 'mean_absolute_error'

In [11]:
from sklearn.metrics import r2_score
import numpy as np

# 生成模拟数据
y_true = np.random.rand(100)
y_pred = np.random.rand(100)

# 将数据分为三个子集
n = len(y_true) // 3
y_true_subsets = [y_true[i:i+n] for i in range(0, len(y_true), n)]
y_pred_subsets = [y_pred[i:i+n] for i in range(0, len(y_pred), n)]

# 分别计算每个子集的R2
r2_subsets = [r2_score(y_true_subsets[i], y_pred_subsets[i]) for i in range(3)]

# 计算所有数据的总体R2
r2_total = r2_score(y_true, y_pred)

# 将三个子集的R2求平均值
r2_subsets_avg = np.mean(r2_subsets)

# 比较结果
print("R2 score for all data: %.4f" % r2_total)
print("Average R2 score for subsets: %.4f" % r2_subsets_avg)
print("Are the results equal? %s" % np.isclose(r2_total, r2_subsets_avg))


R2 score for all data: -1.0849
Average R2 score for subsets: -1.3088
Are the results equal? False


In [493]:
est = linear_model.LinearRegression()
est.fit(np.concatenate((training_x_dynamic,training_x_static),axis = -1), training_y)
print(test_metrics(test_y, est.predict(np.concatenate((test_x_dynamic,test_x_static),axis = -1))))

Test_MAE: 155015.8806
Test_MSE: 24179252143.2908
Test_RMSE: 155496.7914
Test_R2: -284264516.9668
----
None


In [585]:
# Stop training when a monitored metric has stopped improving.监控val_loss,当连续10个epoches，val_loss的变化没有大于0.0001，视为无改进，停止训练，输出进度条
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1)
# Reduce learning rate when a metric has stopped improving.监控val_loss,当连续10个epoches，val_loss的变化没有大于0.001，视为无改进，学习率减半，输出进度条
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=1e-3, factor=0.5, patience=5, verbose=1)
start = time.time()
history = model.fit([training_x_dynamic,training_x_static], training_y, shuffle=True, epochs=150, callbacks=[early_stop, reduce_lr], #
                    validation_data=([val_x_dynamic,val_x_static], val_y), verbose=1, batch_size=20)# val_x_dynamic,val_x_static], val_y
end = time.time()

Epoch 1/150
522/522 [==============================] - 3s 5ms/step - loss: 140.4014 - val_loss: 69.6001
Epoch 2/150
522/522 [==============================] - 2s 4ms/step - loss: 62.0067 - val_loss: 56.0873
Epoch 3/150
522/522 [==============================] - 2s 4ms/step - loss: 55.3468 - val_loss: 57.9348
Epoch 4/150
522/522 [==============================] - 2s 4ms/step - loss: 53.1197 - val_loss: 49.2860
Epoch 5/150
522/522 [==============================] - 2s 4ms/step - loss: 50.2567 - val_loss: 47.8326
Epoch 6/150
522/522 [==============================] - 2s 4ms/step - loss: 47.9625 - val_loss: 45.9058
Epoch 7/150
522/522 [==============================] - 2s 4ms/step - loss: 44.8841 - val_loss: 45.0421
Epoch 8/150
522/522 [==============================] - 2s 4ms/step - loss: 43.3754 - val_loss: 40.8586
Epoch 9/150
522/522 [==============================] - 2s 4ms/step - loss: 40.4264 - val_loss: 38.9600
Epoch 10/150
522/522 [==============================] - 2s 4ms/step - lo

In [67]:
def tune_param(testyear = [2020],MODEL=['LSTM'],verbose=0,shape=False,UNIT=[32],LR = [0.001],BATCH_SIZE=[10],Result=False,seeds=9):
    X_train_val_dynamic,X_train_val_static,X_test_dynamic,X_test_static,y_train_val, y_test,yield_mean, b, slope= yield_predictor(data, data1980, test_year = testyear,  
                                                                                detrend=True, usehistoy=True)
    X_train_dynamic, X_val_dynamic, y_train, y_val = train_test_split(X_train_val_dynamic,y_train_val, test_size=0.15, random_state=seeds)
    X_train_static, X_val_static, y_train, y_val = train_test_split(X_train_val_static,y_train_val, test_size=0.15, random_state= seeds)
    trainshape,valshape,testshape = X_train_dynamic.shape[0],X_val_dynamic.shape[0],X_test_dynamic.shape[0]
    print(testyear)
    if shape:
        print('train:',X_train_dynamic.shape,X_train_static.shape,y_train.shape,
              '\n val:',X_val_dynamic.shape,X_val_static.shape, y_val.shape,
              '\n test:',X_test_dynamic.shape,X_test_static.shape,y_test.shape)
    if MODEL == ['LSTM']:
        # lstm
        training_x_dynamic = np.array(X_train_dynamic).reshape(trainshape,24,38)
        training_x_static = np.array(X_train_static).reshape(trainshape,19)
        training_y = np.array(y_train).reshape(trainshape,1)

        val_x_dynamic = np.array(X_val_dynamic).reshape(valshape,24,38)
        val_x_static = np.array(X_val_static).reshape(valshape,19)
        val_y = np.array(y_val).reshape(valshape,1)

        test_x_dynamic = np.array(X_test_dynamic).reshape(testshape,24,38)
        test_x_static = np.array(X_test_static).reshape(testshape,19)
        test_y = np.array(y_test).reshape(testshape,1)
        ratio = str(trainshape)+':'+str(valshape)+':'+str(testshape)


        for i in UNIT:
            for j in LR:
                for k in BATCH_SIZE:
                    print('UNIT=',i,",LR=",j,',BATCH_SIZE=',k)
                    unit = i
                    inp1 = tf.keras.layers.Input(shape=(24,38))
                    inp2 = tf.keras.layers.Input(shape=(19,))
                    LSTM1 = tf.keras.layers.LSTM(units=unit, return_sequences=True)(inp1)
                    LSTM2 = tf.keras.layers.LSTM(units=unit, return_sequences=False)(LSTM1)
                    # LSTM2 = tf.keras.layers.LSTM(units=unit, return_sequences=True)(LSTM1)
                    # LSTM3 = tf.keras.layers.LSTM(units=unit, return_sequences=True)(LSTM2)
                    # LSTM4 = tf.keras.layers.LSTM(units=32, return_sequences=False)(LSTM1)
                    comb = tf.keras.layers.Concatenate(axis=-1)([LSTM2, inp2])
                    DENSE_1 = tf.keras.layers.Dense(units=unit, activation='relu')(comb)
                    outp = tf.keras.layers.Dense(units=1, activation='linear')(DENSE_1)
                    model = tf.keras.Model([inp1, inp2], outp)
                    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=j), loss=tf.keras.losses.MeanSquaredError())# MeanAbsoluteError
                    early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1)
                    reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=1e-3, factor=0.5, patience=5, verbose=1)# 
#                     start = time.time()
#                     history = model.fit([training_x_dynamic,training_x_static], training_y, shuffle=True, epochs=100, callbacks=[early_stop, reduce_lr], #
#                                         validation_data=([val_x_dynamic,val_x_static], val_y), verbose=1, batch_size=k)# val_x_dynamic,val_x_static], val_y
#                     end = time.time()
                    if verbose == 0:
                        raw,result = model_lineresult(model,MODEL,ratio,testyear,[training_x_dynamic,training_x_static], [val_x_dynamic,val_x_static], 
                                             [test_x_dynamic,test_x_static], training_y, val_y, test_y, k,early_stop, reduce_lr)
                    else :
                        model_result(model,MODEL,ratio,testyear,[training_x_dynamic,training_x_static], [val_x_dynamic,val_x_static], 
                                             [test_x_dynamic,test_x_static], training_y, val_y, test_y, k,early_stop, reduce_lr)
    if MODEL == ['DNN']:
        # lstm
        training_x_dynamic = np.array(X_train_dynamic).reshape(trainshape,912)
        training_x_static = np.array(X_train_static).reshape(trainshape,19)
        training_y = np.array(y_train).reshape(trainshape,1)

        val_x_dynamic = np.array(X_val_dynamic).reshape(valshape,912)
        val_x_static = np.array(X_val_static).reshape(valshape,19)
        val_y = np.array(y_val).reshape(valshape,1)

        test_x_dynamic = np.array(X_test_dynamic).reshape(testshape,912)
        test_x_static = np.array(X_test_static).reshape(testshape,19)
        test_y = np.array(y_test).reshape(testshape,1)
        ratio = str(trainshape)+':'+str(valshape)+':'+str(testshape)

        for i in UNIT:
            for j in LR:
                for k in BATCH_SIZE:
                    print('UNIT=',i,",LR=",j,',BATCH_SIZE=',k)
                    unit = i
                    func_act = 'relu'
                    inp1 = tf.keras.layers.Input(shape=(912))
                    inp2 = tf.keras.layers.Input(shape=(19))
                    inp = tf.keras.layers.Concatenate(axis=-1)([inp1, inp2])
                    DENSE_0 = tf.keras.layers.Dense(units=i*128, activation=func_act)(inp)# ,kernel_regularizer=tf.keras.regularizers.l2(0.001)1028
                    # DENSE_01 = tf.keras.layers.Dropout(0.2)(DENSE_0)
                    DENSE_1 = tf.keras.layers.Dense(units=i*64, activation=func_act)(DENSE_0)#,kernel_regularizer=tf.keras.regularizers.l2(0.001)
                    DENSE_2 = tf.keras.layers.Dense(units=i*32, activation=func_act)(DENSE_1)
                    DENSE_3 = tf.keras.layers.Dense(units=i*16, activation=func_act)(DENSE_2)
                    DENSE_4 = tf.keras.layers.Dense(units=i*8, activation=func_act)(DENSE_3)
                    DENSE_5 = tf.keras.layers.Dense(units=i*4, activation=func_act)(DENSE_4)
                    DENSE_6 = tf.keras.layers.Dense(units=i*2, activation=func_act)(DENSE_5)
                    outp = tf.keras.layers.Dense(units=1, activation='linear')(DENSE_6)
                    model = tf.keras.Model([inp1, inp2], outp)
                    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=j), loss=tf.keras.losses.MeanSquaredError())# 'mean_absolute_error'
                    early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1)
                    reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=1e-3, factor=0.5, patience=5, verbose=1)
#                     start = time.time()
#                     history = model.fit([training_x_dynamic,training_x_static], training_y, shuffle=True, epochs=100, callbacks=[early_stop, reduce_lr], #
#                                         validation_data=([val_x_dynamic,val_x_static], val_y), verbose=1, batch_size=k)# val_x_dynamic,val_x_static], val_y
#                     end = time.time()
                    if verbose == 0:
                        raw,result = model_lineresult(model,MODEL,ratio,testyear,[training_x_dynamic,training_x_static], [val_x_dynamic,val_x_static], 
                                             [test_x_dynamic,test_x_static], training_y, val_y, test_y, k,early_stop, reduce_lr)
                    else :
                        model_result(model,MODEL,ratio,testyear,[training_x_dynamic,training_x_static], [val_x_dynamic,val_x_static], 
                                             [test_x_dynamic,test_x_static], training_y, val_y, test_y, k,early_stop, reduce_lr)
    if Result:
        # print(yield_mean, slope,raw,result)
        # get the yield and pred
        pred = pd.DataFrame([])
#         result = pd.concat([data.loc[test_y.index,['sta_con', 'yield','irrigation','Planted_pdsi','Emergrd_pdsi','Blooming_pdsi','Setting Pods_pdsi',
#                                                    'Dropping Leaves_pdsi','Harvested_pdsi']], pd.DataFrame(result,index = y_test.index,columns=['pred'])],axis=1)
        result1= pd.DataFrame({'yield' : raw.flatten(),
                                'pred' :  result.flatten()},
                                columns=['yield','pred'])
        print(slope,testyear[0],yield_mean,b)
        print(r2_score(raw,result))
        result1.loc[:,'yield'] =result1.loc[:,'yield']+ slope * testyear[0] - yield_mean + b
        result1.loc[:,'pred'] =result1.loc[:,'pred']+ slope * testyear[0] - yield_mean + b
        result1.loc[:,['yield','pred']] = result1.loc[:,['yield','pred']]*62.719012*0.001
        result1.loc[:,'residual'] =  result1.loc[:,'pred'] - result1.loc[:,'yield']
        print(r2_score(result1.loc[:,'yield'],result1.loc[:,'pred']))
        result1.to_csv(r"C:\Users\lx\Desktop\美国产量估计\结果\0614" + str(testyear[0]) + '_LSTM'+str(seeds) + '_预测结果test.csv',header=True)
        print('Results have been saved！')
    return 

In [77]:
for i in [ 2004, 2005, 2010]:#range(2003,2021,1):[2003,2013]:
    tune_param(testyear = [i],MODEL=['LSTM'],verbose=0,shape=False,UNIT=[16],LR = [0.001],BATCH_SIZE=[20],Result=True,seeds=9)

[2004]
UNIT= 16 ,LR= 0.001 ,BATCH_SIZE= 20

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00034: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00046: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00059: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00066: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00071: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 00071: early stopping
2022-06-16 [2004] 10358:1829:852 ['LSTM'] 【Train_time: 278.08, Validation_RMSE: 3.76, Validation_R2: 0.84, Test_RMSE: 6.76, Test_R2: 0.60】
[0.48696257] 2004 45.21594321818331 [-935.12739114]
0.596125098716502
0.5961250987165001
Results have been saved！
[2005]
UNIT= 16 ,LR= 0.001 ,BATCH_SIZE= 20

Epoch 00037: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00060: ReduceLROnPlateau reducing learning rate to 0.000250

In [78]:
for i in [2012]:#range(2003,2021,1):[2003,2013]:
    tune_param(testyear = [i],MODEL=['LSTM'],verbose=0,shape=False,UNIT=[16],LR = [0.001],BATCH_SIZE=[20],Result=True,seeds=99)
    

[2012]
UNIT= 16 ,LR= 0.001 ,BATCH_SIZE= 20

Epoch 00039: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00045: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00051: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00059: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00065: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00075: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00080: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 00080: early stopping
2022-06-16 [2012] 10486:1851:702 ['LSTM'] 【Train_time: 326.49, Validation_RMSE: 3.82, Validation_R2: 0.83, Test_RMSE: 7.53, Test_R2: 0.48】
[0.5033154] 2012 45.31321228823863 [-967.65223448]
0.48498737967490335
0.4849873796749107
Results have been saved！


In [74]:
for i in [2008]:#range(2003,2021,1):[2003,2013]:
    tune_param(testyear = [i],MODEL=['LSTM'],verbose=0,shape=False,UNIT=[16],LR = [0.001],BATCH_SIZE=[20],Result=True,seeds=999)

[2008]
UNIT= 16 ,LR= 0.001 ,BATCH_SIZE= 20

Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00041: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00059: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00074: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00091: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00096: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 00096: early stopping
2022-06-16 [2008] 10491:1852:696 ['LSTM'] 【Train_time: 388.49, Validation_RMSE: 3.77, Validation_R2: 0.84, Test_RMSE: 4.84, Test_R2: 0.60】
[0.49228977] 2008 45.23996597261605 [-945.69792063]
0.6047100620195154
0.6047100620194948
Results have been saved！


In [12]:
# # 保存训练好的模型
# model.save(r'C:\Users\lx\Desktop\美国产量估计\已经训练好的模型\LSTM')
# 读取模型
model = tf.keras.models.load_model(r'C:\Users\lx\Desktop\美国产量估计\已经训练好的模型\LSTM')#\DNN
model.summary()

Model: "functional_358"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_367 (InputLayer)          [(None, 24, 38)]     0                                            
__________________________________________________________________________________________________
lstm_119 (LSTM)                 (None, 16)           3520        input_367[0][0]                  
__________________________________________________________________________________________________
input_368 (InputLayer)          [(None, 19)]         0                                            
__________________________________________________________________________________________________
concatenate_177 (Concatenate)   (None, 35)           0           lstm_119[0][0]                   
                                                                 input_368[0][0]     

In [13]:
# 训练时间
print('训练时间：',end-start)
# 训练评估
y_train_pred = model.predict([training_x_dynamic,training_x_static])
print(train_metrics(training_y, y_train_pred))
# 验证评估
y_val_pred = model.predict([val_x_dynamic,val_x_static])
print(test_metrics(val_y, y_val_pred))
# 测试评估1
y_test_pred = model.predict([test_x_dynamic,test_x_static])
print(test_metrics(test_y, y_test_pred))

print('检查是否过拟合：')
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'end' is not defined

In [14]:

pred_yield = model.predict([test_x_dynamic,test_x_static])
PRED_YIELD = pd.DataFrame(pred_yield, columns=['yield'])
print('linear regression ')
# linear regression 
y_test = np.array(test_y).reshape(1840 ,1)
P_y = np.array(PRED_YIELD['yield']).reshape(1840 ,1)
linear_m=linear_model.LinearRegression()
linear_m.fit(y_test,P_y)

print('plotting\n')
#plot result
plt.figure(figsize=(8,8))
plt.scatter(y_test,P_y,c='k')
plt.plot(y_test,y_test,'r-',label='1:1')
ax = plt.gca()
ax.set_aspect(1)
plt.xlabel('Truth')
plt.ylabel('Predicted')
xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()
X_range=np.arange(xmin, xmax)
Y_range=linear_m.predict(X_range.reshape(len(X_range),1))
plt.plot(X_range,Y_range,'b--',label='y=%.2fx'%(linear_m.coef_[0]))
plt.text(25,55,'$\ R^2$=%.2f'%r2_score(y_test, P_y))
plt.legend()
#plt.savefig(u'/content/drive/MyDrive/csv_data/10_29只用lswi_2层_动态+静态特征'+'.jpg',dpi=500)
plt.show()

NameError: name 'test_x_dynamic' is not defined